# Projekt1 - Beizen der Schweiz

## APIs, Requests, Visualisierung

#### Nenn mir deine Stammbeiz und ich sage dir, wo du wohnst

Welches sind die vorherrschenden Beizennamen pro Kanton? Recherche mit der Swisscom Directories AG API.
Mein persönlicher API Schlüssel: 590ea8961488f9a63dbf95a7b7883805

https://tel.search.ch/api/help


BEISPIEL: https://tel.search.ch/api/?was=john+meier&key=Ihr Schlüssel

In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

%matplotlib inline

import json


In [17]:
"https://tel.search.ch/api/?was=Restaurant&key=590ea8961488f9a63dbf95a7b7883805"



'https://tel.search.ch/api/?was=Restaurant&key=590ea8961488f9a63dbf95a7b7883805'

In [58]:
url = "https://tel.search.ch/api/?was=Restaurant&wo=SH&key=590ea8961488f9a63dbf95a7b7883805"

In [59]:
response = requests.get(url)

In [60]:
response

<Response [200]>

In [61]:
response.text

'<?xml version="1.0" encoding="utf-8" ?>\n<feed xml:lang="de" xmlns="http://www.w3.org/2005/Atom" xmlns:openSearch="http://a9.com/-/spec/opensearchrss/1.0/" xmlns:tel="http://tel.search.ch/api/spec/result/1.0/">\n  <id>https://tel.search.ch/api/590ea8961488f9a63dbf95a7b7883805/8c1dd469af1e26dfc55dd7ea0b982946</id>\n  <title type="text">tel.search.ch API Search Results</title>\n  <generator version="1.0" uri="https://tel.search.ch">tel.search.ch</generator>\n  <updated>2018-10-09T02:00:00Z</updated>\n  <link href="https://tel.search.ch/result.html?was=Restaurant&amp;kanton=SH" rel="alternate" type="text/html" />\n  <link href="http://tel.search.ch/api/?was=Restaurant&amp;wo=SH&amp;key=590ea8961488f9a63dbf95a7b7883805" type="application/atom+xml" rel="self" />\n  <link href="http://tel.search.ch/api/?was=Restaurant&amp;wo=SH&amp;pos=11&amp;key=590ea8961488f9a63dbf95a7b7883805" rel="next" type="application/atom+xml" />\n  <openSearch:totalResults>265</openSearch:totalResults>\n  <openSear

In [62]:
basis = "https://tel.search.ch/api/?"
was = "was=Restaurant&"
wo = "wo=SH&"
key = "key=590ea8961488f9a63dbf95a7b7883805"

In [66]:
url_sh = basis + was + wo + key

In [67]:
requests.get(url_sh)

<Response [200]>

In [68]:
requests.get(url_sh).text

'<?xml version="1.0" encoding="utf-8" ?>\n<feed xml:lang="de" xmlns="http://www.w3.org/2005/Atom" xmlns:openSearch="http://a9.com/-/spec/opensearchrss/1.0/" xmlns:tel="http://tel.search.ch/api/spec/result/1.0/">\n  <id>https://tel.search.ch/api/590ea8961488f9a63dbf95a7b7883805/8c1dd469af1e26dfc55dd7ea0b982946</id>\n  <title type="text">tel.search.ch API Search Results</title>\n  <generator version="1.0" uri="https://tel.search.ch">tel.search.ch</generator>\n  <updated>2018-10-09T02:00:00Z</updated>\n  <link href="https://tel.search.ch/result.html?was=Restaurant&amp;kanton=SH" rel="alternate" type="text/html" />\n  <link href="http://tel.search.ch/api/?was=Restaurant&amp;wo=SH&amp;key=590ea8961488f9a63dbf95a7b7883805" type="application/atom+xml" rel="self" />\n  <link href="http://tel.search.ch/api/?was=Restaurant&amp;wo=SH&amp;pos=11&amp;key=590ea8961488f9a63dbf95a7b7883805" rel="next" type="application/atom+xml" />\n  <openSearch:totalResults>265</openSearch:totalResults>\n  <openSear

In [69]:
print(url_sh)

https://tel.search.ch/api/?was=Restaurant&wo=SH&key=590ea8961488f9a63dbf95a7b7883805


In [75]:
requests.get(url_sh).headers

{'Server': 'Apache', 'Vary': 'Accept-Encoding', 'Content-Type': 'text/xml; charset=utf-8', 'Content-Encoding': 'gzip', 'Date': 'Tue, 09 Oct 2018 17:44:02 GMT', 'Access-Control-Allow-Origin': '*', 'Connection': 'Keep-Alive', 'Set-Cookie': 'myosotis=dcccdc6bece11028f4e94621823eb388; expires=Tue, 19-Jan-2038 03:14:07 GMT; Max-Age=608376605; path=/; domain=.search.ch; HttpOnly', 'Content-Length': '2631'}

In [78]:
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [84]:
r = requests.get(url_sh)

In [87]:
contents = r.text #Wandeln wir den Text in ein Format um, mit dem BeautifulSoup umgehen kann.
soup = BeautifulSoup(contents,'html.parser') #Geben wir das an BeautifulSoup weiter
titles = soup.find_all('title') #Nun lesen wir Titel aus.
len(titles) #Schauen wir, wie lange die Titel sind.

11

In [114]:
print (titles[1])

<title type="text">Güterhof Restaurant, Bar, Sushi, Dine</title>


In [115]:
(titles[1]).text

'Güterhof Restaurant, Bar, Sushi, Dine'

In [119]:
descr = soup.find_all('title')
descr[1]

<title type="text">Güterhof Restaurant, Bar, Sushi, Dine</title>